In [ ]:
from datetime import datetime
import calendar
import pandas as pd
import matplotlib.pyplot as plt

# Set matplotlib to run inline so we can view visualization
%matplotlib inline

# Import and call the Riak client to connect to your Riak TS node or cluster
# See http://docs.basho.com/riak/ts/latest/developing/python/ for more information
# on how to use the Riak client for Python
from riak import RiakClient
# Note: If you wish Riak TS to return date fields as Python dates
# and not epoch you need to set the transport_options as shown below
client = RiakClient(transport_options={'ts_convert_timestamp': True})

table = "WaterMeterMonthlyRollUp"

In [ ]:
# Function to convert Python date to Unix Epoch
def convert_to_epoch ( date_to_convert ):
    return calendar.timegm(datetime.timetuple( date_to_convert )) * 1000

In [ ]:
# Function to convert TsObject to list of lists
def ts_obj_to_list_of_lists (ts_obj):
    list_to_return = []
    for row in ts_obj.rows:
        list = []
        for i in range(len(row)):
            list.append(row[i])
        list_to_return.append(list)
    return list_to_return

In [ ]:
# Retrieve all rows in WaterMeterMonthlyRollUp
start_date = datetime(2016, 1, 1, 0, 00)
end_date = datetime(2017, 2, 1, 0, 00)

query = """\
    SELECT *
    FROM WaterMeterMonthlyRollUp
    WHERE reading_date >= {} AND reading_date < {} AND
        customer_id = 'CUSTOMER-0001' AND meter_id = 'METER-0001'
""".format( convert_to_epoch( start_date ), convert_to_epoch( end_date ) )

data_set = client.ts_query(table, query)

for row in data_set.rows:
    print row

In [ ]:
# Convert TsObject to list of lists and create Pandas DataFrame
boring_list = ts_obj_to_list_of_lists( data_set )
df = pd.DataFrame( boring_list )

# Set the column names and the index to the Date field
df.columns = ['Customer','Meter','Date','Month','Year', 'Gallons', 'Bill']
df.set_index('Date', False, False, True, True)

In [ ]:
df

In [ ]:
df['Gallons']

In [ ]:
df['20160301':'20160601']

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

In [ ]:
df.info()

In [ ]:
df.memory_usage()

In [ ]:
print df.describe()

In [ ]:
df.count()

In [ ]:
df.min()

In [ ]:
df.max()

In [ ]:
df.median()

In [ ]:
df.sort_values(by='Bill', ascending=False)

In [ ]:
df.groupby('Gallons').sum()

In [ ]:
df.plot()